In [38]:
import pandas as pd
import numpy as np

from functools import reduce

In [49]:
data = pd.read_csv('test_data\\2018.csv', header=0)

In [50]:
data.columns

Index(['1_period_our_goals', '1_period_our_shots', '1_period_their_goals',
       '1_period_their_shots', '2_period_our_goals', '2_period_our_shots',
       '2_period_their_goals', '2_period_their_shots', '3_period_our_goals',
       '3_period_our_shots', '3_period_their_goals', '3_period_their_shots',
       'away', 'blocked', 'faceOffWinPercentage', 'game_id', 'giveaways',
       'goals', 'hits', 'id', 'pim', 'powerPlayGoals',
       'powerPlayOpportunities', 'powerPlayPercentage', 'shots', 'takeaways',
       'OT'],
      dtype='object')

In [35]:
def add_win_col(data):
    data['our_goals'] = data['1_period_our_goals'] + data['2_period_our_goals'] + data['3_period_our_goals']
    data['their_goals'] = data['1_period_their_goals'] + data['2_period_their_goals'] + data['3_period_their_goals']
    data['win'] = (data['our_goals'] > data['their_goals']).astype('int')
    return data

In [55]:
# whole period features

def get_win_rate(data):
    wins_home = data[~data['away']].groupby('id')['win'].sum()
    games_home = data[~data['away']].groupby('id').size()
    winrate_home = (wins_home / games_home).rename('win_rate_home').reset_index()
    wins_away = data[data['away']].groupby('id')['win'].sum()
    games_away = data[data['away']].groupby('id').size()
    winrate_away = (wins_away / games_away).rename('win_rate_away').reset_index()
    return winrate_home.merge(winrate_away, on='id')

def get_avg_periods_goals(features, data):
    for period in [1, 2, 3]:
        avg_home_period = data[~data['away']].groupby('id')['{}_period_our_goals'.format(period)].mean().rename('{}_period_avg_home'.format(period)).reset_index()
        features = features.merge(avg_home_period)
    for period in [1, 2, 3]:
        avg_away_period = data[data['away']].groupby('id')['{}_period_our_goals'.format(period)].mean().rename('{}_period_avg_away'.format(period)).reset_index()
        features = features.merge(avg_away_period)
    return features

In [56]:
# calculate team features

data = add_win_col(data)
team_features = get_win_rate(data)
team_features = get_avg_periods_goals(team_features, data)

In [57]:
team_features

,id,win_rate_home,win_rate_away,1_period_avg_home,2_period_avg_home,3_period_avg_home,1_period_avg_away,2_period_avg_away,3_period_avg_away
0,1,0.608696,0.173913,1.217391,1.173913,1.260870,0.869565,0.782609,0.826087
1,2,0.409091,0.363636,0.954545,0.909091,0.909091,0.772727,1.000000,0.818182
2,3,0.578947,0.052632,1.000000,1.368421,1.684211,0.789474,1.105263,0.736842
3,4,0.409091,0.454545,1.318182,1.227273,0.909091,0.954545,1.181818,1.045455
4,5,0.350000,0.300000,0.650000,1.200000,0.850000,0.850000,1.200000,1.050000
5,6,0.375000,0.333333,0.750000,0.916667,1.000000,0.875000,0.750000,0.708333
6,7,0.391304,0.304348,1.130435,1.130435,0.695652,0.782609,1.043478,0.956522
7,8,0.350000,0.400000,0.800000,1.350000,0.650000,1.050000,0.850000,0.800000
8,9,0.700000,0.250000,1.150000,1.800000,1.450000,0.800000,0.800000,1.100000
9,10,0.260870,0.565217,1.000000,0.913043,1.086957,0.956522,1.347826,1.434783
